In [5]:
import os
import pm4py

log = pm4py.read_xes(os.path.join("data", "BPI_Challenge_2017.xes"))

frequency_edges = pm4py.discover_network_analysis(log, out_column="case:concept:name", in_column="case:concept:name", node_column_source="org:resource", node_column_target="org:resource", edge_column="concept:name", performance=False)
pm4py.view_network_analysis(frequency_edges, variant="frequency", format="svg", edge_threshold=10)

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

c:\users\fabian harmsen\pycharmprojects\praxismodul1_pm4py\venv\lib\site-packages\pm4py\algo\organizational_mining\network_analysis\variants\dataframe.py:167: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dataframe = dataframe[{timestamp_column, in_column, out_column, node_column_source, node_column_target, edge_column, sorting_column}]


KeyboardInterrupt: 